# NBA Salary Prediction Based on Age of player

This notebook demonstrates how to build a simple machine learning model to predict the salary of an NBA player based on the number of years they have played in the league.

## Step 1: Import Libraries and Load the Data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import streamlit as st

import snowflake.snowpark.types as t
import snowflake.snowpark.functions as f

from snowflake.snowpark import DataFrame

from snowflake.ml.registry import Registry
from snowflake.ml.modeling.linear_model import LinearRegression
from snowflake.ml.modeling.neural_network import MLPRegressor
import snowflake.ml.modeling.metrics as metrics
from snowflake.ml.modeling.preprocessing import StandardScaler


# create Snowflake session so we can work with data
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
-- use schema snowpark_playground.nba;
-- ls @SNOWFLAKE_NOTEBOOK_ML_101/branches/main;

In [ ]:
-- select $1 from @snowpark_playground.nba.snowflake_notebook_ml_101/branches/main/nba.csv;

-- CREATE FILE FORMAT if not exists my_csv_format
--   TYPE = csv
--   PARSE_HEADER = true;

-- SELECT *
--   FROM TABLE(
--     INFER_SCHEMA(
--       LOCATION=>'@snowflake_notebook_ml_101/branches/main/nba.csv'
--       , FILE_FORMAT=>'my_csv_format'
--       )
--     );

In [ ]:
# df_nba = session.read.option("INFER_SCHEMA", True).option("PARSE_HEADER", True).csv("@snowflake_notebook_ml_101/branches/main/nba.csv")

# df_nba.show()

In [ ]:
# import pandas as pd
# df = pd.read_csv("nba.csv")
# df

In [ ]:
# Load the nba.csv dataset from Snowflake, for instance using Snowsight
# .... loading....done.

# assuming this is where the data was loaded
# change this if your data is elsewhere
data = session.table('SNOWPARK_PLAYGROUND.NBA.SALARIES')

# Preview the data
data.show()

## Step 2: Data Exploration and Visualization

In [ ]:
# check the schema
list(data.schema)

In [ ]:
# Basic statistics
st.write(data.describe())


# Visualize the relationship between age and salary
plt.scatter(x='AGE', y='SALARY', data=data.to_pandas())
plt.xlabel('AGE')
plt.ylabel('SALARY')
plt.title('Age vs Salary')
plt.show()

## Step 3: Prepare the Data

In [ ]:
# Split the data into training and testing sets
data_train, data_test = data.random_split(weights=[0.8, 0.2], seed=0)

# cast so we don't get complaints
# optional but you'll get warning later if you don't do it
data_train=\
data_train.select([f.col("AGE").cast('double').as_('AGE'),f.col("SALARY").cast('double').as_('SALARY')])

data_test=\
data_test.select([f.col("AGE").cast('double').as_('AGE'),f.col("SALARY").cast('double').as_('SALARY')])

st.write(data_train.describe())
list(data_train.schema)


In [ ]:
#let's count missing values (rows) per column
def null_columns(df: DataFrame):
    total_records = df.count()
    for column in df.columns:
        not_null_count = f.count(column)
        df = df.withColumn(column, f.lit(total_records - not_null_count))
    return df.limit(1)

st.write(null_columns(data_train))
st.write(null_columns(data_test))

In [ ]:
# drop NaNs
data_train = data_train.dropna()
data_test = data_train.dropna()


# Check for missing values again - good to go?
st.write(null_columns(data_train))
st.write(null_columns(data_test))

## Step 4: Train the Model

In [ ]:
# care to comment?
features=['AGE']
labels=['SALARY']

In [ ]:
# Create a linear regression model
model = LinearRegression(
    input_cols=features,
    label_cols=labels
)

# Fit the model
model.fit(data_train)

# Output the linear reg. model coefficients (have to explicitely convert to sklearn for this)
model_local=model.to_sklearn()
print(f'Intercept: {model_local.intercept_}')
print(f'Coefficient: {model_local.coef_[0]}')

## Step 5: Evaluate the Linear Regression Model

In [ ]:
# Make predictions on the test set
pred = model.predict(data_test)

In [ ]:
# check out the result
pred

In [ ]:

# Evaluate the model
mse = metrics.mean_squared_error(df=pred,y_true_col_names=['SALARY'], y_pred_col_names=['OUTPUT_SALARY'])
r2=metrics.r2_score(df=pred,y_true_col_name='SALARY', y_pred_col_name='OUTPUT_SALARY')

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

pred=pred.sort(f.col('AGE').desc())

y_pred = np.array(pred.select('OUTPUT_SALARY').collect())
x_pred= np.array(pred.select('AGE').collect())

plt.scatter(x='AGE',y='SALARY',data=data_test.to_pandas(), color='blue')
plt.plot(x_pred, y_pred, color='red', linewidth=2)
plt.xlabel('AGE')
plt.ylabel('SALARY')
plt.title('Linear Regression: Age vs Salary')
plt.show()

## Step 6: Make a prediction for age of 25

In [ ]:
# Predict the salary for a player with a certain age
player_age=25
age = session.create_dataframe([player_age], schema=['AGE'])
predicted_salary = model.predict(age)
predicted_salary

## Step 7. Fit a Neural Network Model

In [ ]:
# Scale the data (standardization)
scaler = StandardScaler(
    input_cols=['AGE'],
    output_cols=['AGE_SCALED']
)

scaler.fit(data_test)
data_train_scaled = scaler.transform(data_train)
data_test_scaled=scaler.transform(data_test)


# Create the MLPRegressor model
# Try playing with the parameters!
mlp = MLPRegressor(
    input_cols=['AGE_SCALED'],
    label_cols=labels,
    hidden_layer_sizes=(64, 64), activation='relu', solver='lbfgs', max_iter=500, random_state=1)

# Fit the model
res= mlp.fit(data_train_scaled)

# Run predictions
pred_mlp=mlp.predict(data_test_scaled)

## Step 8. Evaluate a NNM

In [ ]:
# check the output
pred_mlp

In [ ]:
# Let's evaluate

data_test=data_test.sort(f.col('AGE').desc())
data_sorted=data.sort(f.col('AGE').desc())

pred_mlp=pred_mlp.sort(f.col('AGE_SCALED').desc())


mse_mlp = metrics.mean_squared_error(df=pred_mlp,y_true_col_names=['SALARY'], y_pred_col_names=['OUTPUT_SALARY'])
r2_mlp=metrics.r2_score(df=pred_mlp,y_true_col_name='SALARY', y_pred_col_name='OUTPUT_SALARY')

print(f'Mean Squared Error: {mse_mlp}')
print(f'R^2 Score: {r2_mlp}')

x_test= np.array(data_test.select('AGE').collect())
y_pred = np.array(pred_mlp.select('OUTPUT_SALARY').collect())

# Visualize the results
plt.scatter(x='AGE',y='SALARY',data=data_sorted.to_pandas(), color='blue', label='Actual')
# plt.scatter(X_test, y_test, color='blue', label='Actual')
plt.plot(x_test, y_pred, color='red', label='Predicted')
plt.xlabel('Age')
plt.ylabel('Salary')
plt.title('MLPRegressor: Age vs Salary')
plt.legend()
plt.show()


## Step 9. Make a prediction for the age of 25

In [ ]:
# Predict the salary for a player with a certain number of years played
age = session.create_dataframe([player_age], schema=['AGE'])

age_scaled = scaler.transform(age)

predicted_salary = mlp.predict(age_scaled)
predicted_salary



## Step 10. Register models

In [ ]:
-- model registry
create schema if not exists snowpark_playground.registry

In [ ]:
# create a Snowflake Registry object
reg = Registry(session=session, database_name="SNOWPARK_PLAYGROUND", schema_name="REGISTRY")

In [ ]:
# Register the Linear Regression model
mv = reg.log_model(model,
                   model_name="nba_salary_lin_reg",
                   # conda_dependencies=["scikit-learn"],
                   comment="A LinearRegression ML model that predicts NBA salaries",
                   metrics={"r2": r2},
                   # sample_input_data=data_train,
                   options = {'relax_version':False}
                  )

#what did we get back?
mv

In [ ]:
reg.log_model(mlp,
             model_name="nba_salary_mlp",
             comment = "A MLPRegressor ML model for predicting NBA salaries",
             metrics={"r2":r2_mlp,"mse":mse_mlp},
            options = {'relax_version':False}
            )

In [ ]:
# Check the models in the registry
reg.show_models()

## 11. Use a registered model to create predictions on data in Snowflake 

In [ ]:
# Create a view that will be used in SQL to predict the target variable
data_test_scaled.create_or_replace_temp_view('snowpark_playground.nba.INFERENCE_TEST')

In [ ]:
select * from snowpark_playground.nba.inference_test limit 10;

In [ ]:
use schema snowpark_playground.registry;

-- call predict from the latest model version
WITH latest AS MODEL NBA_SALARY_MLP VERSION LAST
    SELECT AGE, (latest!predict(AGE_SCALED)):OUTPUT_SALARY::double as salary_prediction FROM snowpark_playground.nba.inference_test;

## Summary
This notebook provides a simple implementation of a linear regression model and MLPRegressor model to predict NBA salaries based on the age of the player. It also shows how to work with a Model Registry in Snowflake and how to use a registered model in SQL to run a prediction on a Snowflake table or view. 
Depending on the available data, the model could be enhanced by incorporating additional features or by using more complex models.
